### Import Library

In [18]:
import os
import cv2
# from skimage.color import rgb2gray
# from PIL import Image, ImageOps
# import skimage.io as io
import numpy as np
import pandas as pd
import math

In [8]:
# Data normal dan glaukoma
image_folder_path ={
    'togray':
    {
        'otsu':
        {
            'r2':
            {
                'normal' : r".\dataset\R2\toGray\otsu\normal",
                'glaukoma': r".\dataset\R2\toGray\otsu\glaukoma",
            },
            'r3':
            {
                'normal' : r".\dataset\R3\toGray\otsu\normal",
                'glaukoma': r".\dataset\R3\toGray\otsu\glaukoma",
            },
            'r2r3':
            {
                'normal' : r".\dataset\r2r3\otsu\normal",
                'glaukoma': r".\dataset\r2r3\otsu\glaukoma",
            }                
        },
        'canny':
        {
            'r2':
            {
                'normal' : r".\dataset\R2\toGray\canny\normal",
                'glaukoma': r".\dataset\R2\toGray\canny\glaukoma",
            },
            'r3':
            {
                'normal' : r".\dataset\R3\toGray\canny\normal",
                'glaukoma': r".\dataset\R3\toGray\canny\glaukoma",
            },
            'r2r3':
            {
                'normal' : r".\dataset\r2r3\canny\normal",
                'glaukoma': r".\dataset\r2r3\canny\glaukoma",
            }                
        }
    },
}

### GLCM

In [2]:
def GLCM00(dataset):
    #membentuk matriks dengan ordo nxn dengan n adalah nilai elemen terbesar dari matriks
    a,b = np.shape(dataset)
    matriks = np.zeros((np.max(dataset)+1,np.max(dataset)+1))  
    maxnilai = np.max(dataset)+1 #nilai elemen terbesar dari matriks
    for i in range(a):
        for j in range(b-1):
            matriks[dataset[i][j],dataset[i][j+1]] = matriks[dataset[i][j],dataset[i][j+1]]+1
            matriks[dataset[i][j+1],dataset[i][j]] = matriks[dataset[i][j+1],dataset[i][j]]+1
    return matriks

def GLCM45(dataset):
    a,b = np.shape(dataset)
    matriks = np.zeros((np.max(dataset)+1,np.max(dataset)+1))
    maxnilai = np.max(dataset)+1
    for i in range(a-1):
        for j in range(b-1):
            matriks[dataset[i+1][j],dataset[i][j+1]] = matriks[dataset[i+1][j],dataset[i][j+1]]+1
            matriks[dataset[i][j+1],dataset[i+1][j]] = matriks[dataset[i][j+1],dataset[i+1][j]]+1
    return matriks

def GLCM90(dataset):
    a,b = np.shape(dataset)
    matriks = np.zeros((np.max(dataset)+1,np.max(dataset)+1))
    maxnilai = np.max(dataset)+1
    for i in range(a-1):
        for j in range(b):
            matriks[dataset[i+1][j],dataset[i][j]] = matriks[dataset[i+1][j],dataset[i][j]]+1
            matriks[dataset[i][j],dataset[i+1][j]] = matriks[dataset[i][j],dataset[i+1][j]]+1
    return matriks

def GLCM135(dataset):
    a,b = np.shape(dataset)
    matriks = np.zeros((np.max(dataset)+1,np.max(dataset)+1))
    maxnilai = np.max(dataset)+1
    for i in range(a-1):
        for j in range(b-1):
            matriks[dataset[i+1][j+1],dataset[i][j]] = matriks[dataset[i+1][j+1],dataset[i][j]]+1
            matriks[dataset[i][j],dataset[i+1][j+1]] = matriks[dataset[i][j],dataset[i+1][j+1]]+1
    return matriks

def contrast(dataset):
    kontras=0
    for i in range(len(dataset)):
        for j in range(len(dataset)):
            kontras+=(i-j)**2*dataset[i][j]
    return kontras

def IDM(dataset):
    idm=0
    for i in range(len(dataset)):
        for j in range(len(dataset)):
            idm+=1/(1+(i-j)**2)*dataset[i][j]
    return idm

def entropy(dataset):
    entropi=0
    for i in range(len(dataset)):
        for j in range(len(dataset)):
            if dataset[i,j] != 0.0:
                entropi+=-dataset[i][j]*math.log(dataset[i][j],10)
    return entropi

def correlation(dataset):
    korelasi=0
    M=0
    T=0
    for i in range(len(dataset)):
        M+=i*dataset[i]
    M2 = sum(M)
    for i in range(len(dataset)):
        T+=dataset[i]*(i-M2)**2
    T2=sum(T)
    for i in range(len(dataset)):
        for j in range(len(dataset)):
            korelasi+=dataset[i][j]*(i-M2)*(j-M2)/T2**2
    return korelasi

def energy(dataset):
    energi=0
    for i in range(len(dataset)):
        for j in range(len(dataset)):
            energi+=dataset[i][j]**2
    return energi

In [3]:
#Normalisasi Matriks GLCM
def NormalizeGLCM(dataset):
    return dataset/sum(sum(dataset))

### Statistik Berbasis Histogram

In [4]:
#rerata intensitas
def intensitas_histogram(dataset):
    #hitung frekuensi aras keabuan
    L = np.max(dataset)+2
    m,n = np.shape(dataset)
    Frek = np.zeros((L,1))
    grayimg = dataset.astype(float)
    for i in range(1,m) :
        for j in range(1,n) :
            intensitas = grayimg[i,j]
            Frek[int(intensitas+1)] = Frek[int(intensitas+1)]+1

    #hitung probabilitas
    jum_piksel = m * n
    Prob = np.zeros((L,1))
    for i in range(0,(L-1)):
        Prob[i+1]= Frek[i+1]/jum_piksel
    
    #hitung rerata intensitas
    mu = 0
    for i in range(0,(L-1)):
        #print(Prob)
        mu = mu + i * Prob[i+1]
    mu = mu[0]
    return mu
        
#deviasi standar
def standar_dev_histogram(dataset):
    #hitung frekuensi aras keabuan
    L = np.max(dataset)+2
    m,n = np.shape(dataset)
    Frek = np.zeros((L,1))
    grayimg = dataset.astype(float)
    for i in range(1,m) :
        for j in range(1,n) :
            intensitas = grayimg[i,j]
            Frek[int(intensitas+1)] = Frek[int(intensitas+1)]+1

    #hitung probabilitas
    jum_piksel = m * n
    Prob = np.zeros((L,1))
    for i in range(0,(L-1)):
        Prob[i+1]= Frek[i+1]/jum_piksel
        
    #hitung deviasi
    varians = 0
    mu = intensitas_histogram(dataset)
    for i in range(0,(L-1)):
        varians = varians + (i-mu)**2 * Prob[i+1]
    deviasi = math.sqrt(varians)
    varians_n = varians / (L-1)**2 #normalisasi
    varians_n = varians_n[0]
    return varians_n

#skewness
def skewness_histogram(dataset):
    #hitung frekuensi aras keabuan
    L = np.max(dataset)+2
    m,n = np.shape(dataset)
    Frek = np.zeros((L,1))
    grayimg = dataset.astype(float)
    for i in range(1,m) :
        for j in range(1,n) :
            intensitas = grayimg[i,j]
            Frek[int(intensitas+1)] = Frek[int(intensitas+1)]+1

    #hitung probabilitas
    jum_piksel = m * n
    Prob = np.zeros((L,1))
    for i in range(0,(L-1)):
        Prob[i+1]= Frek[i+1]/jum_piksel
        
    #menghitung skewness
    skewness = 0
    mu = intensitas_histogram(dataset)
    for i in range(0,(L-1)):
        skewness = skewness + (i-mu)**3 * Prob[i+1]
    skewness = skewness / (L-1)**2
    skewness = skewness[0]
    return skewness

#energi
def energi_histogram(dataset):
    #hitung frekuensi aras keabuan
    L = np.max(dataset)+2
    m,n = np.shape(dataset)
    Frek = np.zeros((L,1))
    grayimg = dataset.astype(float)
    for i in range(1,m) :
        for j in range(1,n) :
            intensitas = grayimg[i,j]
            Frek[int(intensitas+1)] = Frek[int(intensitas+1)]+1

    #hitung probabilitas
    jum_piksel = m * n
    Prob = np.zeros((L,1))
    for i in range(0,(L-1)):
        Prob[i+1]= Frek[i+1]/jum_piksel
        
    #menghitung energi
    energi = 0
    for i in range(0,(L-1)):
        energi = energi + Prob[i+1]**2
    energi = energi[0]
    return energi

#entropi
def entropi_histogram(dataset):
    #hitung frekuensi aras keabuan
    L = np.max(dataset)+2
    m,n = np.shape(dataset)
    Frek = np.zeros((L,1))
    grayimg = dataset.astype(float)
    for i in range(1,m) :
        for j in range(1,n) :
            intensitas = grayimg[i,j]
            Frek[int(intensitas+1)] = Frek[int(intensitas+1)]+1

    #hitung probabilitas
    jum_piksel = m * n
    Prob = np.zeros((L,1))
    for i in range(0,(L-1)):
        Prob[i+1]= Frek[i+1]/jum_piksel
        
    #menghitung entropi
    entropi = 0
    for i in range(0,(L-1)):
        if(Prob[i+1] != 0):
            entropi = entropi + Prob[i+1] * math.log(Prob[i+1])
    entropi = -entropi
    entropi = entropi[0]
    return entropi
    
#kehalusan
def kehalusan_histogram(dataset):
    varians_n = standar_dev_histogram(dataset)
    smoothness = 1 - (1 / (1+varians_n))
    return smoothness

### Membuat rangka data csv

In [5]:
def CreateDataFrame():
    return {
        'filename' : [],
        'contrast_00' : [],
        'contrast_45' : [],
        'contrast_90' : [],
        'contrast_135' : [],
        'IDM_00' : [],
        'IDM_45' : [],
        'IDM_90' : [],
        'IDM_135' : [],
        'entropy_00' : [],
        'entropy_45' : [],
        'entropy_90' : [],
        'entropy_135' : [],
        'corellation_00' : [],
        'corellation_45' : [],
        'corellation_90' : [],
        'corellation_135' : [],
        'energy_00' : [],
        'energy_45' : [],
        'energy_90' : [],
        'energy_135' : [],
        'intensitas_histo' : [],
        'standar_dev_histo' : [],
        'skewness_histo' : [],
        'energi_histo' : [],
        'entropi_histo' : [],
        'kehalusan_histo' : [],
        
        'class' : [],
    }
dataframe = CreateDataFrame()

In [6]:
#Main Ekstraksi
def HitungFitur(grayscale_images_folder_path, penyakit):
    for filename in os.listdir(grayscale_images_folder_path):
        filename2 = filename
        filename = os.path.join(grayscale_images_folder_path, filename)

        img = cv2.imread(filename, 0)

        dataframe['filename'].append(filename2)
        dataframe['contrast_00'].append(contrast(NormalizeGLCM(GLCM00(img))))
        dataframe['contrast_45'].append(contrast(NormalizeGLCM(GLCM45(img))))
        dataframe['contrast_90'].append(contrast(NormalizeGLCM(GLCM90(img))))
        dataframe['contrast_135'].append(contrast(NormalizeGLCM(GLCM135(img))))
        dataframe['IDM_00'].append(IDM(NormalizeGLCM(GLCM00(img))))
        dataframe['IDM_45'].append(IDM(NormalizeGLCM(GLCM45(img))))
        dataframe['IDM_90'].append(IDM(NormalizeGLCM(GLCM90(img))))
        dataframe['IDM_135'].append(IDM(NormalizeGLCM(GLCM135(img))))
        dataframe['entropy_00'].append(entropy(NormalizeGLCM(GLCM00(img))))
        dataframe['entropy_45'].append(entropy(NormalizeGLCM(GLCM45(img))))
        dataframe['entropy_90'].append(entropy(NormalizeGLCM(GLCM90(img))))
        dataframe['entropy_135'].append(entropy(NormalizeGLCM(GLCM135(img))))
        dataframe['corellation_00'].append(correlation(NormalizeGLCM(GLCM00(img))))
        dataframe['corellation_45'].append(correlation(NormalizeGLCM(GLCM45(img))))
        dataframe['corellation_90'].append(correlation(NormalizeGLCM(GLCM90(img))))
        dataframe['corellation_135'].append(correlation(NormalizeGLCM(GLCM135(img))))
        dataframe['energy_00'].append(energy(NormalizeGLCM(GLCM00(img))))
        dataframe['energy_45'].append(energy(NormalizeGLCM(GLCM45(img))))
        dataframe['energy_90'].append(energy(NormalizeGLCM(GLCM90(img))))
        dataframe['energy_135'].append(energy(NormalizeGLCM(GLCM135(img))))
        dataframe['intensitas_histo'].append(intensitas_histogram(img))
        dataframe['standar_dev_histo'].append(standar_dev_histogram(img))
        dataframe['skewness_histo'].append(skewness_histogram(img))
        dataframe['energi_histo'].append(energi_histogram(img))
        dataframe['entropi_histo'].append(entropi_histogram(img))
        dataframe['kehalusan_histo'].append(kehalusan_histogram(img))
    
    
        #img = cv2.imread(filename)
        dataframe['class'].append(penyakit)

### CSV Otsu

In [19]:
#Ekstraks Fitur Training (Membuat dataframe untuk dataset r2)
dataframe = CreateDataFrame()

HitungFitur(image_folder_path['togray']['otsu']['r2r3']['normal'],0)
HitungFitur(image_folder_path['togray']['otsu']['r2r3']['glaukoma'],1)

dataframe = pd.DataFrame(data=dataframe)
dataframe.to_csv('Otsu r2r3 dataset.csv', index=False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in double_scalars
